In [1]:
import boto3

In [2]:
# S3 클라이언트 생성
s3 = boto3.client('s3')

# S3 버킷 이름과 업로드 경로 설정
bucket_name = 'smwu-cv-team-2-s3'  # 사용할 S3 버킷 이름
model_path = 'models/best_model.pth'  # S3에 저장할 경로
local_model_file = 'best_model_34.pth'  # SageMaker에서 저장한 로컬 모델 파일 경로

# 로컬 파일을 S3로 업로드
s3.upload_file(local_model_file, bucket_name, model_path)
print(f"Model uploaded to s3://{bucket_name}/{model_path}")

Model uploaded to s3://smwu-cv-team-2-s3/models/best_model.pth


In [3]:
from sagemaker.pytorch import PyTorchModel

# SageMaker에서 모델 생성
pytorch_model = PyTorchModel(
    model_data=f"s3://smwu-cv-team-2-s3/models/best_model.pth",  # S3 모델 파일 경로
    role='arn:aws:iam::730335373015:role/service-role/AmazonSageMaker-ExecutionRole-20240815T144726',  # SageMaker 실행 역할(Role)
    entry_point='inference.py',  # 추론 스크립트 경로 (아래에서 작성할 예정)
    framework_version='1.9.1',
    py_version='py38'
)

print("SageMaker Model created successfully!")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
SageMaker Model created successfully!


In [2]:
from sagemaker import image_uris

# 필요한 설정
region = "us-east-1"  # SageMaker 노트북이 실행 중인 지역
framework = "pytorch"
version = "1.13.1"  # PyTorch 버전 (학습 시 사용한 버전)
py_version = "py39"  # Python 버전
instance_type = "ml.g4dn.2xlarge"  # GPU 인스턴스 타입

# GPU 서빙 컨테이너 이미지 URI 가져오기
image_uri = image_uris.retrieve(
    framework=framework,
    region=region,
    version=version,
    py_version=py_version,
    instance_type=instance_type,
    image_scope="inference"  # 서빙용 컨테이너
)

print(f"PyTorch Inference Image URI: {image_uri}")

PyTorch Inference Image URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.13.1-gpu-py39


In [3]:
import boto3

s3 = boto3.client('s3')
bucket_name = "smwu-cv-team-2-s3"  # 버킷 이름
inference_file = "inference.py"
inference_path = "scripts/inference.py"  # S3에 저장할 경로

# 파일 업로드
s3.upload_file(inference_file, bucket_name, inference_path)
print(f"inference.py uploaded to s3://{bucket_name}/{inference_path}")

inference.py uploaded to s3://smwu-cv-team-2-s3/scripts/inference.py


In [6]:
import tarfile

# .tar.gz 파일 생성
with tarfile.open("best_model_34.tar.gz", "w:gz") as tar:
    tar.add("best_model_34.pth", arcname="best_model_34.pth")

print("Model file compressed to best_model_34.tar.gz")

Model file compressed to best_model_34.tar.gz


In [ ]:
###############################

In [11]:
import boto3

s3 = boto3.client('s3')
bucket_name = "smwu-cv-team-2-s3"
s3_model_path = "models/final7_model.tar.gz"

# S3로 파일 업로드
s3.upload_file('model.tar.gz', bucket_name, s3_model_path)
print(f"File uploaded to s3://{bucket_name}/{s3_model_path}")

File uploaded to s3://smwu-cv-team-2-s3/models/final7_model.tar.gz


In [12]:
from sagemaker.pytorch import PyTorchModel

# S3 경로 설정
bucket_name = 'smwu-cv-team-2-s3'  # S3 버킷 이름
s3_key = 'models/final7_model.tar.gz'  # S3에서 파일 경로
model_data = f"s3://{bucket_name}/{s3_key}"

# SageMaker Role 설정
role = 'arn:aws:iam::730335373015:role/service-role/AmazonSageMaker-ExecutionRole-20240815T144726'

# PyTorch 모델 생성
pytorch_model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',  
    framework_version='1.13.1',
    py_version='py39'
)

print("Model created successfully!")

Model created successfully!


In [13]:
# 엔드포인트 배포
predictor = pytorch_model.deploy(
    initial_instance_count=1,  # 엔드포인트 인스턴스 수
    instance_type='ml.g4dn.2xlarge',  # 인스턴스 유형
    endpoint_name='34-endpoint'  # 엔드포인트 이름
)

print(f"Endpoint deployed: {predictor.endpoint_name}")

----------!Endpoint deployed: 34-endpoint
